# Ollama RAG Chatbot 1
Retrieval-augmented generation is a technique that allows Large Language Models (LLMs) to retrieve and incorporate new information.  

## Project Objective
The objective of this project is to develop a Retrieval-Augmented Generation (RAG) chatbot using LlamaIndex to assist employees in accessing accurate and timely information about Capital One’s 2025 benefits offerings. The chatbot will leverage a Large Language Model (LLM) enhanced by an external knowledge base, the 2025 Capital One Benefits PDF document, to retrieve and generate relevant answers to employee inquiries.

## Reference
- Local-LLM-Chatbot-using-Ollama-and-Langchain, M-A-S1
- Reading Documents Directly from Llama Index as Files Instead of Specifying a Folder Path, Stackoverflow 78338438
- Claude: Llamaindex RAG
- Build a chatbot with customer data sources, powered by Llamaindex, Caroline Frasca, Krista Muir, and Yi Ding

## Setup

In [ ]:
# Install llama-index-llms-ollama
!pip install llama-index-llms-ollama

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 27.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 12.4 MB/s eta 0:00:00
  Attempting uninstall: setuptools
    Found existing installation: setuptools 75.2.0
    Uninstalling setuptools-75.2.0:
      Successfully uninstalled setuptools-75.2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


In [ ]:
# Install Ollama
!sudo apt update
!sudo apt install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [3,103 kB]
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,266 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/

In [ ]:

# Running Ollama

import threading
import subprocess
import time

def run_ollama_serve():
  subprocess.Popen(["ollama", "serve"])

thread = threading.Thread(target=run_ollama_serve)
thread.start()
time.sleep(5)

In [ ]:
# Pulling LLM

!ollama pull llama3.2

In [ ]:
# Pull embedding model
!ollama pull nomic-embed-text

In [ ]:
# List available models
!ollama list

NAME                       ID              SIZE      MODIFIED       
nomic-embed-text:latest    0a109f422b47    274 MB    2 seconds ago     
llama3.2:latest            a80c4f17acd5    2.0 GB    11 minutes ago    


In [ ]:
# load Ollama
from llama_index.llms.ollama import Ollama

In [ ]:
llm = Ollama(
    model="llama3.2",
    request_timeout=300.0,
    # Manually set the context window to limit memory usage
    context_window=8000,
)

## Test LLM

In [ ]:
# Test Model
resp = llm.complete("How many oceans are there?")

In [ ]:
# Print Reply
print(resp)

There is one global ocean, which is often referred to as the "Pacific Ocean" or simply "the ocean." However, it's generally considered to be divided into five distinct bodies of water:

1. Pacific Ocean
2. Atlantic Ocean
3. Indian Ocean
4. Arctic Ocean
5. Southern Ocean (also known as the Antarctic Ocean)

The International Hydrographic Organization (IHO) recognized the Southern Ocean as a separate ocean in 2000, defining it as the waters surrounding Antarctica and extending north to the coast of South America, Africa, and Australia.

So, while there's only one global ocean, it's often divided into these five distinct bodies of water for geographical, scientific, and practical purposes.


## Prep for RAG

In [ ]:
# install llama-index, llama-index-reader-web. cohere, jedi
!pip install llama-index llama-index-readers-web cohere jedi

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 4.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 89.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of llama-cloud-services to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 291.9/291.9 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 71.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Install llama-index-embeddings-ollama
!pip install llama-index-embeddings-ollama

In [ ]:
# Format output
import textwrap

## RAG Creation and Testing

In [ ]:
# Retrieval-augmented Generation
# Use the 2025 Capital One Beneftis document as a knowledge base for the LLM


# Load Libraries
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.ollama import OllamaEmbedding

# Identify embedding model
embed_model = OllamaEmbedding(model_name="nomic-embed-text")

# LLM was previously identified

# Set global settings
Settings.embed_model = embed_model
Settings.llm = llm


# Load the Benefits document
reader = SimpleDirectoryReader(input_files = ["/2025_Benefits_Guide_capitalone.pdf"])
text = reader.load_data()
print(f"Length of docs: {len(text)}")

# Split the document into chunks
text_splitter = SentenceSplitter(chunk_size=1000, chunk_overlap=200)
docs = text_splitter.get_nodes_from_documents(text)

# Create the Index.
# The embedding step is automatic when using Llamaindex
# Llamaindex has a built-in vector store
index = VectorStoreIndex(docs)

# chat engine
chat_engine = index.as_chat_engine(chat_mode="condense_question")

# Questions

prompt = input("Your question: ")
response = chat_engine.chat(prompt)
print(response.response)

Length of docs: 34
Your question: can you summarize capitalone benefits
Capital One offers a comprehensive Total Rewards package that includes various benefits to support your total well-being. These benefits cover medical and prescription drug, dental and vision coverage, as well as financial protection options to help you save for the future. The guide provides an overview of these benefits, but it's essential to review all available options to maximize value for you and your family.


In [ ]:
prompt = input("Your question: ")
response = chat_engine.chat(prompt)
print("Chatbot: ",textwrap.fill(response.response,width=100))

Your question: Does capitalone offer health insurance to its employees
Chatbot:  Capital One's Total Rewards package includes a comprehensive healthcare component that offers
medical, dental, and vision coverage.


In [ ]:
prompt = input("Your question: ")
response = chat_engine.chat(prompt)
print("Chatbot: ",textwrap.fill(response.response,width=100))

Your question: when is an employee eligible for health, dental, and vision benefits
Chatbot:  Yes, you can be eligible for Capital One's medical, dental, and vision benefits if you enroll by the
date of hire.


In [ ]:
prompt = input("Your question: ")
response = chat_engine.chat(prompt)
print("Chatbot: ",textwrap.fill(response.response,width=100))

Your question: Does capitalone offer tax-advantaged spending and savings accounts
Chatbot:  Yes, Capital One offers a Health Savings Account (HSA) as a tax-advantageous spending and savings
option. It provides triple tax benefits: contributions are made pre-tax, the account balance earns
interest tax-free, and withdrawals for eligible medical expenses are also tax-free.


## Next Steps
- Develop a user-friendly interface for the chatbot
- Determine deployment strategy